# OAlley API IPython example

This tutorial will show you how to use the [OAlley API](https://api.oalley.fr) with a few python modules to compute multiple isochronous zones efficiently.

Unlike the previous tutorial, this tutorial performs requests against the OAlley server asynchronously.
It does not wait for a request to complete to perform the next one, making you more productive.

You will need to install python packages `folium` `grequests` and  `polyline`.
Easy:
```
pip install folium grequests polyline
```

Also, you will need an API key. Go to [api.oalley.fr](https://api.oalley.fr) and signup. It's free. Each new user receives a key with 1000 requests available.

Copy your key and paste it in the code below.

In [31]:
import folium
import grequests
import polyline
from urllib.parse import urlencode

baseurl = "https://api.oalley.fr/api/AppKeys/"
method = "/isochronous?"

appkey = "YOUR-APP-KEY"


## Call the API

Here, we compute isochronous areas of 1min to 3min around the [Arc de Triomphe](http://tools.wmflabs.org/geohack/geohack.php?language=fr&pagename=Arc_de_triomphe_de_l%27%C3%89toile&params=48.8738_N_2.295_E_) in Paris.

In [29]:
points = [
    {
        "lat":48.8738,
        "lng":2.295,
        "duration": 60 # 1 min by car
    },
    {
        "lat":48.8738,
        "lng":2.295,
        "duration":120 # 2 min
    },
    {
        "lat":48.8738,
        "lng":2.295,
        "duration":180 # 3 min
    },
    
]

# Build all request URLs
urls = [baseurl + appkey + method + urlencode(point) for point in points]

zones = []

def callback(res, **kwargs):
    
    if(res.status_code != 200):
        # Most likely, you don't have credits anymore. 
        # We give more for free ;) Contact us at contact@oalley.fr
        return print(res.json()) 
    
    body = res.json()
    zone = polyline.decode(body['polyline'])
    zones.append(zone)
    
def exception_handler(request, exception):
    print("Error : %s" % exception)
    
# Prepare all requests
reqs = [grequests.get(url, callback=callback) for url in urls]

# Calling map fires all requests at once, and wait until they are completed
grequests.map(reqs, exception_handler=exception_handler)

zones

[[(48.87395, 2.3001),
  (48.87474, 2.29984),
  (48.87566, 2.29956),
  (48.87487, 2.2966),
  (48.87649, 2.29777),
  (48.87666, 2.29713),
  (48.8757, 2.29572),
  (48.87439, 2.29512),
  (48.8745, 2.29492),
  (48.87449, 2.29492),
  (48.87436, 2.29477),
  (48.8755, 2.29369),
  (48.87529, 2.29351),
  (48.87484, 2.29344),
  (48.87549, 2.29032),
  (48.8745, 2.29152),
  (48.87365, 2.28872),
  (48.87305, 2.29164),
  (48.87077, 2.28793),
  (48.87172, 2.29249),
  (48.87046, 2.2912),
  (48.87087, 2.29309),
  (48.87049, 2.29384),
  (48.87038, 2.29423),
  (48.87162, 2.29513),
  (48.87036, 2.29577),
  (48.87183, 2.29553),
  (48.87111, 2.29697),
  (48.87124, 2.29727),
  (48.87271, 2.29663),
  (48.87265, 2.29839),
  (48.87295, 2.29832)],
 [(48.87407, 2.30442),
  (48.87571, 2.30466),
  (48.87846, 2.30625),
  (48.87931, 2.30313),
  (48.8785, 2.3001),
  (48.88192, 2.30033),
  (48.87806, 2.2968),
  (48.87804, 2.29551),
  (48.87788, 2.29538),
  (48.87793, 2.29417),
  (48.88023, 2.29238),
  (48.88172, 2.28899

## Print the results

We will now use a folium map to print the isochronous areas on a map.

In [30]:
# Build output map
m = folium.Map(location=[46, 2], zoom_start=5)

# Draw all computed zones on the map
for points in zones:
    # Trick to close the polyline, until folium library supports it
    points.append(points[0])
    p = folium.PolyLine(locations=points,weight=3)
    m.add_children(p)

# Print the result
m.fit_bounds(m.get_bounds())

# IPython trick to display the map
m

## What's next

See the next example on how to use open-datasets in conjunction with OAlley (TBD).

![compute-all.jpg](./compute-all-iso.jpg)